In [6]:
!pip install transformers scipy ftfy accelerate

In [7]:
pip install --upgrade diffusers huggingface_hub

In [8]:
import torch
from diffusers import StableDiffusionPipeline

image_pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
image_pipe = image_pipe.to("cuda")

In [10]:
from diffusers import AudioLDMPipeline
import torch

model_id = "cvssp/audioldm-s-v2"
audio_pipe = AudioLDMPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
audio_pipe.to("cuda");

In [12]:
generator = torch.Generator("cuda").manual_seed(0)

In [11]:
import soundfile as sf  # To save audio in WAV format
from IPython.display import Audio

# Combined workflow
prompt = "A peaceful forest with birds chirping and a flowing stream"

# Generate Image
image = image_pipe(prompt).images[0]
image.save("generated_image.png")

# Generate Audio
audio = audio_pipe(prompt, num_inference_steps=10, audio_length_in_s=5, generator=generator).audios[0]

# Save the generated audio to a WAV file
sf.write("generated_audio.wav", audio, 16000)  # Save at 16000 Hz
print("Image and audio generated successfully!")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Image and audio generated successfully!


In [13]:
# Play the audio in the notebook

Audio("generated_audio.wav", rate=16000, autoplay=True)


In [13]:
pip install gradio

In [14]:
import gradio as gr
import soundfile as sf

def generate_image_and_audio(prompt):
    try:
        # Generate Image
        print("Generating image...")
        image = image_pipe(prompt).images[0]
        print("Image generated successfully.")

        # Generate Audio
        print("Generating audio...")
        audio = audio_pipe(prompt, num_inference_steps=10, audio_length_in_s=5).audios[0]
        print("Audio generated successfully.")

        # Save Audio
        audio_path = "generated_audio.wav"
        sf.write(audio_path, audio, samplerate=16000)
        print("Audio saved successfully.")

        return image, audio_path
    except Exception as e:
        print("An error occurred:", e)
        return None, None

# Gradio Interface
interface = gr.Interface(
    fn=generate_image_and_audio,
    inputs="text",
    outputs=["image", "audio"]
)

# Launch with debugging enabled
interface.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f2f987d8fc367252f5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generating image...


  0%|          | 0/50 [00:00<?, ?it/s]

Image generated successfully.
Generating audio...


  0%|          | 0/10 [00:00<?, ?it/s]

Audio generated successfully.
Audio saved successfully.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://f2f987d8fc367252f5.gradio.live
